# Multispecies Coalescent (species trees)

### Learning objectives:
- Multispecies Coalescent is stitching together of multiple populations.
- Incomplete lineage sorting is coalescent events deeper than a speciation event.
- The difference between a true genealogy and inferred gene tree.

In [28]:
import toytree
import ipcoal

## Species tree (population structure)

The Multispecies Coalescent (MSC) is a model that describes genealogical relationships among 2 or more populations (or species). You can think of this model as a "stitching together" of coalescent models for multiple distinct populations. This is sometimes also called a "species tree". A species tree is defined by a topology (branching order) with edge lengths (in units of generations), and effective population sizes assigned to each edge. This effectively defines a container in which genealogies are embedded. Remember, this is the topic we introduced at the beginning of notebook 1, that genealogies are embedded within species trees.

If we view the coalescent process as occurring backwards in time, from the tips towards the root of the species tree, we can follow the coalescence of samples within each interval (branch) of the species tree. Starting from the tips of the species tree, gene copies can coalesce within the species tree edge in which they originate, or not. If they do not coalesce by the time they reach the top of that species tree interval, they enter the next ancestral interval (an internal species tree edge) at which point they can potentially coalesce with gene copies from the other species tree branch that was also descended this interval. For example, gene copies that do not coalesce within species tree edge 0 below, will enter species tree edge 3, at which point they might coalesce with a gene copy from population 1 before coalescing with another sample from population 0. This is an example of a process termed **incomplete lineage sorting**. This is would lead to the genealogy not matching the topology of the species tree.

Once again, let's use simulations to visualize this process.

In [30]:
# first define a species tree topology and draw it
species_tree = toytree.rtree.imbtree(ntips=3, treeheight=100_000)
species_tree.draw('p');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t01416bf335a74fac86eff6ec5f2a2e25"> 0 1 2 3 4 r0 r1 r2 0 50000 100000

### Low `Ne` (much smaller than species tree edge lengths)

Similar to the last notebook we will simulate multiple gene copies within a population, but here we simulate this process for three populations related by the species tree topology. In the scenario below, we will set a fixed `Ne` value for all edges of the species tree, here setting it to a very small value. The coalescences happen very quickly within each population because the `Ne` value is so low relative to the species tree edge lengths. 

The common ancestor of the samples in population "r0" cannot coalesce with the common ancestor of samples in the "r1" population until farther back in time than the speciation event that separates them. We can see in the species tree above that this event is at time=50,000. This is also highlighted in the genealogy plot below using a dashed line. You can see that these ancestors coalesce very rapidly as soon as they enter this same ancestral species tree branch, soon after it becomes possible for them to coalesce. 

The same thing then happens again with the common ancestor of populations "r0" and "r1" and the common ancestor "r2" very soon after time=100,000. 

In [31]:
# simulate three populations, each with low Ne
model = ipcoal.Model(tree=species_tree, Ne=1_000, nsamples=4, seed_trees=123)
model.sim_trees(4)
c, a, m = model.draw_genealogies(shared_axes=True, scale_bar=True, node_sizes=0, tip_labels_style={'font-size': 9});

# add a horizontal line at time=50K
[i.hlines(50_000, color='green', style={'stroke-dasharray': "2,4"}) for i in a];
[i.hlines(100_000, color='green', style={'stroke-dasharray': "2,4"}) for i in a];

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="750.0px" height="250.0px" viewBox="0 0 750.0 250.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1c87e8e0c97f491184a3daa3d6ed0279"> r2_1 r2_3 r2_0 r2_2 r1_0 r1_3 r1_1 r1_2 r0_2 r0_0 r0_1 r0_3 0 50000 100000 r2_0 r2_1 r2_2 r2_3 r1_1 r1_2 r1_0 r1_3 r0_3 r0_1 r0_0 r0_2 0 50000 100000 r2_3 r2_0 r2_1 r2_2 r0_0 r0_3 r0_1 r0_2 r1_2 r1_3 r1_0 r1_1 0 50000 100000 r2_0 r2_1 r2_2 r2_3 r1_2 r1_0 r1_1 r1_3 r0_2 r0_3 r0_0 r0_1 0 50000 100000

### larger `Ne` (relative to species tree edge lengths)

In the example above each population forms a monophyletic clade, since the samples coalesce very rapidly. However, when `Ne` is larger this may not be the case. If the time between  coalescent events become longer than the length of the species tree edges then coalesce events may occur deeper in time than the earliest interval in which samples can coalesce. When this occurs it can lead to incomplete lineage sorting (ILS). 

In this example, the samples in each species only sometimes form monophyletic clades. For example, in the first genealogy below, sample "r0_2" groups with the samples in population "r1" instead of with the other samples in population "r0". This is inly possible because sample "r0_2" persisted farther back in time than the speciation event at time=50,000 before coalescing with another sample. In this ancestral interval it then happened to coalesce with the population "r0" samples before it did with samples from "r1". This is an example of **incomplete lineage sorting**.

In [57]:
# simulate three populations, each with higher Ne
model = ipcoal.Model(tree=species_tree, Ne=10_000, nsamples=4, seed_trees=123)
model.sim_trees(4)
c, a, m = model.draw_genealogies(shared_axes=True, scale_bar=True, node_sizes=0, tip_labels_style={'font-size': 9});

# add a horizontal line at time=50K, 100K
[i.hlines(50_000, color='green', style={'stroke-dasharray': "2,4"}) for i in a];
[i.hlines(100_000, color='green', style={'stroke-dasharray': "2,4"}) for i in a];

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="750.0px" height="250.0px" viewBox="0 0 750.0 250.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te39d2a247ec84d45a2e997530623246e"> r2_1 r2_3 r2_0 r2_2 r0_0 r0_1 r0_3 r0_2 r1_0 r1_3 r1_1 r1_2 0 50000 100000 r2_0 r2_1 r2_2 r2_3 r1_1 r1_2 r1_0 r1_3 r0_3 r0_1 r0_0 r0_2 0 50000 100000 r2_3 r2_0 r2_1 r2_2 r0_0 r1_2 r0_3 r0_1 r0_2 r1_3 r1_0 r1_1 0 50000 100000 r2_0 r2_1 r2_2 r2_3 r0_0 r0_1 r0_2 r0_3 r1_2 r1_0 r1_1 r1_3 0 50000 100000

## Simulating sequences
Now let's go a step farther to simulate the genealogies and simulate mutations occurring on top of them. We will simulate a multilocus dataset, similar in style to a RAD-seq data set. We can treat each locus as representing an independent genealogy on which mutations were added. Here we simulate a small dataset of 100 loci, each 50 sites in lenght, for the same 3 population tree as above, using Ne=10K for each population, and sampling 4 samples per population.

In [58]:
# add arguments for the mutation rate and a seed for the random generator
model = ipcoal.Model(
    tree=species_tree, 
    Ne=10_000, 
    nsamples=4,
    mut=1e-7,
    recomb=0,
    seed_trees=123,
    seed_mutations=333,
    store_tree_sequences=True,  # allows us to plot the mutations below
)

# call sim_loci to generate independent loci of a given length
model.sim_loci(nloci=109, nsites=50)

### View summary of simulation
Now that we have a `ipcoal.Model` object named `model` storing our simulation, we can view a summary of the simulated data. This table shows the 100 loci, their lengths, the number of SNPs, and the genealogy for each one.

In [59]:
model.df

,locus,start,end,nbps,nsnps,tidx,genealogy
0,0,0,50,50,4,0,(((r2_1:1633.12885320076...
1,1,0,50,50,2,0,((r2_0:10331.51915627964...
2,2,0,50,50,4,0,((r2_3:33008.79873577909...
3,3,0,50,50,1,0,((r2_0:23353.31180242069...
4,4,0,50,50,4,0,((r2_2:45706.46330074605...
...,...,...,...,...,...,...,...
104,104,0,50,50,2,0,(r1_2:108764.05393018730...
105,105,0,50,50,0,0,((r2_0:16384.12726912045...
106,106,0,50,50,2,0,(((r2_2:5918.34923995394...
107,107,0,50,50,2,0,((r2_0:10240.10632984724...


### View genealogy and sequences at locus=0
We can see in the table above that locus 0 contains 4 SNPs. We can also see these SNPs visually in the drawing below of the simulated sequence matrix, where four sites contain variation and all of the other sites in this 50 bp locus are invariant.

Let's now compare the sequences to the genealogy below it. We can see clearly in this example how mutations arising on the genealogy lead to the observed sequence variation. For example, three of the samples in population "r1" form a clade to the exclusion of the other "r1" sample. It appears two mutations occurred on this branch, which is why sample "r1_0" is different from the other "r1" samples at these two sites.

Try to figure out which other mutations plotted on the branches of the genealogy below are associated with which sites in the sequence matrix. Next, edit the number assigned to the variable `LOCUS` below to select a different locus to compare the sequences and genealogy, selecting any number between 0 and 99.

In [60]:
# select a locus number (0-99)
LOCUS = 0

# draw the sequences and genealogy for the selected locus
model.draw_seqview(idx=LOCUS, show_text=True, scrollable=False);
model.draw_genealogy(idx=LOCUS, show_substitutions=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1060.0px" height="292.0px" viewBox="0 0 1060.0 292.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0529d716a46d49d2aca2e8c4b31c6e7b"> r0_0 T G T T G T A T A C T G T G G T G A C G C T C C A T A A A T C A T C T C T C C C C G T C G T C C A T r0_1 T G T T G T A T A C T G T G G T G A C G C T C C A T A A A T C A T C T C T C C C C G T C G T C C A T r0_2 T G T T T T A T A C T G T G G T G A C G C T C C A T A A A T C A T C T C T C C C C G T C G T C C A T r0_3 T G T T G T A T A C T G T G G T G A C G C T C C A T A A A T C A T C T C T C C C C G T C G T C C A T r1_0 T G T T T T A T A C T G T G G T G A C G C T C C A T A A A T C A T C T C T C C C C G T C G T C C A T r1_1 T G T T T T G T A G T G T G G T G A C G C T C C A T A A A T C A T C T C T C C C C G T C G T C C A T r1_2 T G T T T T G T A G T G T G G T G A C G C T C C A T A A A T C A T C T C T C C C C G T C G T C C A T r1_3 T G T T T T G T A G T G T G G T G A C G C T C C A T A A A T C A T C T C T C C C C G T C G T C C A T r2_0 T G T T T T A T A C T G A G G T G A C G C T C C A T A A A T C A T C T C T C C C C G T C G T C C A T r2_1 T G T T T T A T A C T G T <rect x="338.0999999999999" y="195.125" width="16.69999999999999" height="14.625" style="f

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="298.71999999999997px" viewBox="0 0 300.0 298.71999999999997" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tb641da43895b460684169b5f05142436"> r2_1 r2_3 r2_0 r2_2 r0_0 r0_1 r0_3 r0_2 r1_0 r1_3 r1_1 r1_2 id: 0
site: 0
time: 82027
mtype: 0 id: 1
site: 1
time: 21967
mtype: 0 id: 2
site: 2
time: 18424
mtype: 0 id: 3
site: 3
time: 25861
mtype: 0 100000 50000 0 tree in interval 0 (sites 0-50)

## Genealogy versus Gene Tree

A very important take home message from this exercise to understand the difference between the true **genealogy** for a set of samples, and the **gene tree** that can inferred from the sequence variation that arose on a genealogy. The latter is an estimate, and is often limited by the amount of information. Below, we infer a gene tree using raxml from the sequences at each locus. 

In [61]:
# infers a raxml tree at each locus
raxml_data = ipcoal.phylo.infer_raxml_ng_trees(model)

In [62]:
# this is a dataframe where the 'gene_tree' column contains the inferred trees
raxml_data

,locus,start,end,nbps,nsnps,gene_tree
0,0,0,50,50,4,"(r0_3:1e-06,r0_1:1e-06,(..."
1,1,0,50,50,2,"(r2_0:1e-06,r2_3:1e-06,r..."
2,2,0,50,50,4,"(r0_0:0.020494,r2_2:1e-0..."
3,3,0,50,50,1,"(r0_0:1e-06,r2_3:1e-06,r..."
4,4,0,50,50,4,"((r2_3:1e-06,r2_1:1e-06,..."
...,...,...,...,...,...,...
104,104,0,50,50,2,"(r1_0:1e-06,(r0_1:1e-06,..."
105,105,0,50,50,0,"(r0_0:1e-06,r2_3:1e-06,r..."
106,106,0,50,50,2,"(r2_3:1e-06,r2_2:1e-06,r..."
107,107,0,50,50,2,"(r2_3:1e-06,r2_2:1e-06,r..."


### Individual gene trees
Because there is limited information in each locus (similar to a RAD locus) each gene tree contains only a limited amount of resolution. Here we show the first 4 inferred gene trees in the data set.

In [63]:
# draw a few inferred trees
toytree.mtree(raxml_data.gene_tree).draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="750.0px" height="250.0px" viewBox="0 0 750.0 250.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ted13108b93834018a8d53dccbee0bdb5"> r0_3 r0_1 r1_1 r1_3 r1_2 r2_2 r2_0 r1_0 r2_3 r2_1 r0_2 r0_0 r2_0 r2_3 r2_2 r0_0 r0_1 r1_0 r0_3 r1_1 r1_3 r0_2 r1_2 r2_1 r0_0 r2_2 r1_1 r2_0 r1_3 r1_2 r0_1 r0_3 r0_2 r2_3 r2_1 r1_0 r0_0 r2_3 r2_2 r0_3 r0_2 r2_1 r0_1 r1_1 r1_2 r1_3 r1_0 r2_0

### Concatenation tree
This is one of the reasons that we often infer a concatenation tree. That is, we combine all of the sequences together to infer one tree that will represent the most dominant signal across all of the data. We can also infer a concatenated tree for the sequences from the 100 simulated loci here. We can see that this looks like a pretty accurate tree, matching with our expectation of the known species tree.

In [79]:
# infer the concatenation tree, root it, and draw it.
concat_tree = ipcoal.phylo.infer_raxml_ng_tree(model, nboots=100)
concat_tree = concat_tree.root("~r2")
concat_tree.draw(ts='r', tip_labels_align=True, node_labels="support");

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="362.2px" viewBox="0 0 300.0 362.2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6bd5972365114cadbac4951704de78ab"> 69 34 100 73 70 100 59 42 100 100 nan r2_0 r2_1 r2_3 r2_2 r1_0 r1_1 r1_3 r1_2 r0_2 r0_3 r0_1 r0_0

### Astral species tree

In [80]:
astral_tree = ipcoal.phylo.infer_astral_tree(
    trees=raxml_data.gene_tree, 
    binary_path="/home/deren/mambaforge-pypy3/envs/py39/bin/astral.5.7.1.jar",
)

In [81]:
astral_tree = astral_tree.root("~r2")
astral_tree.draw(ts='r', tip_labels_align=True, node_labels="support");

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="362.2px" viewBox="0 0 300.0 362.2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc2673525202a4d79baa2fae2d0faea0b"> 0.78 0.42 1 0.52 0.42 1 0.4 0.56 1 1 nan r2_3 r2_2 r2_0 r2_1 r1_0 r1_1 r1_3 r1_2 r0_1 r0_2 r0_0 r0_3